In [5]:
import nest_asyncio
from crewai import LLM
from dotenv import load_dotenv
import os
import asyncio
from echo.constants import (
    DISCOVERY,
    DEMO,
    PRICING,
    NEGOTIATION
)


nest_asyncio.apply()

load_dotenv()

llm = LLM(
    model=os.getenv("FIREWORKS_MODEL_NAME"),
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)

In [6]:
from echo.runner import create_or_get_seller

inputs = {
    "seller": "Whatfix",
    "num_buyers": 10,
}

seller = asyncio.run(create_or_get_seller(inputs))

Creating seller for Whatfix
Adding Seller: Whatfix Data


In [7]:
from echo.runner import make_call


buyer_inputs = {
    **inputs,
    **seller,
    'n_competitors': 10,
}

clients = seller['seller_clients']['clients']

In [8]:
seller['seller_clients']['clients']

['Cardinal Health',
 'Marketboomer',
 'PlayOJO',
 'Sophos Firewall',
 'Reed',
 'Internet Brands']

In [9]:
discovery_calls_data = asyncio.run(make_call(DISCOVERY, seller['seller_clients']['clients'], buyer_inputs))

Making discovery call for ['Cardinal Health', 'Marketboomer', 'PlayOJO', 'Sophos Firewall', 'Reed', 'Internet Brands']
Getting analysis Data
Number of Clients:  6


Getting Data:   0%|          | 0/6 [00:00<?, ?it/s]2025-02-12 00:37:04,142 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Getting Data for Cardinal Health
Data not found for client: Cardinal Health
Data not found for client: Cardinal Health
Adding Buyer: Cardinal Health Data


2025-02-12 00:37:21,685 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding discovery Transcript Data to Vector Store


...

2025-02-12 00:40:44,062 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding discovery Transcript Data to Vector Store


2025-02-12 00:40:54,172 - 8645349184 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Adding Analysis Data to Vector Store


Getting Data: 100%|██████████| 6/6 [03:56<00:00, 39.43s/it]


In [16]:
demo_calls_data = asyncio.run(make_call(DEMO, clients, buyer_inputs))

Making demo call for ['Cardinal Health', 'Marketboomer', 'PlayOJO', 'Sophos Firewall', 'Reed', 'Internet Brands']
Getting analysis Data
Number of Clients:  6


Getting Data:  17%|█▋        | 1/6 [00:00<00:00,  5.25it/s]

Getting Data for Cardinal Health
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Cardinal Health', 'call_type': 'demo', 'company_size': 'Enterprise', 'industry': 'Healthcare', 'description': 'Cardinal Health improves the cost-effectiveness of healthcare. We help focus on patient care while reducing costs, enhancing efficiency and improving ...'}
Getting Data for Marketboomer
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Marketboomer', 'call_type': 'demo', 'company_size': 'SMB', 'industry': 'Hospitality', 'description': 'A cloud-based digital procurement software company serving the hospitality industry'}
Getting Data for PlayOJO


Getting Data: 100%|██████████| 6/6 [00:00<00:00, 24.07it/s]

Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'PlayOJO', 'call_type': 'demo', 'company_size': 'Mid-Market (11-50 employees)', 'industry': 'Gambling Facilities and Casinos', 'description': 'PlayOJO is a UK-based online casino that offers a wide range of games, including slots, blackjack, roulette, and live casino games.'}
Getting Data for Sophos Firewall
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Sophos Firewall', 'call_type': 'demo', 'company_size': 'Enterprise', 'industry': 'Cybersecurity', 'description': 'Sophos Limited is a British security software and hardware company. It develops and markets managed security services and cybersecurity software and hardware.'}
Getting Data for Reed
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Reed', 'call_type': 'demo', 'company_size': 'Enterprise', 'industry': 'Recruitment', 'description

In [17]:
pricing_call_data = asyncio.run(make_call(PRICING, clients, buyer_inputs))

Making pricing call for ['Cardinal Health', 'Marketboomer', 'PlayOJO', 'Sophos Firewall', 'Reed', 'Internet Brands']
Getting analysis Data
Number of Clients:  6


Getting Data: 100%|██████████| 6/6 [00:00<00:00, 54.76it/s]

Getting Data for Cardinal Health
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Cardinal Health', 'call_type': 'pricing', 'company_size': 'Enterprise', 'industry': 'Healthcare', 'description': 'Cardinal Health improves the cost-effectiveness of healthcare. We help focus on patient care while reducing costs, enhancing efficiency and improving ...'}
Getting Data for Marketboomer
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Marketboomer', 'call_type': 'pricing', 'company_size': 'SMB', 'industry': 'Hospitality', 'description': 'A cloud-based digital procurement software company serving the hospitality industry'}
Getting Data for PlayOJO
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'PlayOJO', 'call_type': 'pricing', 'company_size': 'Mid-Market (11-50 employees)', 'industry': 'Gambling Facilities and Casinos', 'description': 'PlayOJO i

In [18]:
negotiation_call_data = asyncio.run(make_call(NEGOTIATION, clients, buyer_inputs))

Making negotiation call for ['Cardinal Health', 'Marketboomer', 'PlayOJO', 'Sophos Firewall', 'Reed', 'Internet Brands']
Getting analysis Data
Number of Clients:  6


Getting Data: 100%|██████████| 6/6 [00:00<00:00, 141.90it/s]

Getting Data for Cardinal Health
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Cardinal Health', 'call_type': 'negotiation', 'company_size': 'Enterprise', 'industry': 'Healthcare', 'description': 'Cardinal Health improves the cost-effectiveness of healthcare. We help focus on patient care while reducing costs, enhancing efficiency and improving ...'}
Getting Data for Marketboomer
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'Marketboomer', 'call_type': 'negotiation', 'company_size': 'SMB', 'industry': 'Hospitality', 'description': 'A cloud-based digital procurement software company serving the hospitality industry'}
Getting Data for PlayOJO
Adding Analysis Data to Vector Store
Node already exists in index: {'seller': 'Whatfix', 'buyer': 'PlayOJO', 'call_type': 'negotiation', 'company_size': 'Mid-Market (11-50 employees)', 'industry': 'Gambling Facilities and Casinos', 'description'

## Queries

In [13]:
# len(index._storage_context.vector_store._collection.get()['metadatas'])

In [1]:
from echo.queries import get_queries

seller = "Whatfix"
queries = get_queries(seller)

In [2]:
print("Total queries: ", len(queries))

Total queries:  4


In [21]:
queries.keys()

dict_keys(['discovery', 'demo', 'pricing', 'negotiation'])

In [30]:
from typing import List
from llama_index.core.schema import NodeWithScore
from echo.indexing import get_vector_index
from echo.query_executor import get_metadata_filters

seller, buyer, call_type = "Whatfix", "Sophos", "buyer_research"

def get_buyer_research(call_type: str, seller: str, buyer: str) -> str:
    metadata_filters = get_metadata_filters(call_type, {"buyer": buyer, "company_size": "Enterprise"})  # noqa: F821
    vector_index = get_vector_index(seller, call_type)
    retriever = vector_index.as_retriever(filters=metadata_filters)
    docs: List[NodeWithScore] = retriever.retrieve("")
    assert len(docs) > 0, f"No Buyer Research documents found for {buyer}"
    return docs[0].text

Buyer Research Fields Description: class ClientResearchResponse(BaseModel):

	name: str # The name of the client.
	website: str # The website of the buyer.
	description: str # A description of the buyer.
	industry: str # The industry that the buyer belongs to.
	company_size: str # The size of the buyer's company as one of the types - SMB, Mid-Market, Enterprise.
	goals: List[str] # The goals of the buyer.
	use_cases: List[str] # The use cases of the buyer.
	challenges: List[str] # The challenges faced by the buyer.
	stakeholders: List[str] # The stakeholders of the buyer.

# Name
Sophos
# Website
https://www.sophos.com/en-us
# Description
Sophos is a British security software and hardware company that develops and markets managed security services and cybersecurity software and hardware.
# Industry
Cybersecurity
# Company_Size
Enterprise
# Goals
   - To be the best in the world at delivering complete IT security to small, mid-market enterprises and the channel that serves them.
   - To

In [ ]:
# vector_index.storage_context.vector_store._collection.get()['metadatas']

In [2]:
from echo.query_executor import arun_queries
from echo.query_executor import ResponseFormat
from echo.query_executor import ContextExtractionMode
import nest_asyncio
import asyncio

nest_asyncio.apply()


inputs = {"buyer": "Sophos", "company_size": "Enterprise"}

responses = asyncio.run(
    arun_queries(
        queries,
        inputs=inputs,
        response_format=ResponseFormat.MARKDOWN,
        context_extraction_mode=ContextExtractionMode.QUERY_ENGINE,
    )
)

Running queries for call type discovery


Running Queries:   0%|          | 0/8 [00:00<?, ?it/s]

Running sub query What all pieces of information about the buyer are uncovered in discovery calls of successful deals?
Sub query context {'query': 'What all pieces of information about the buyer are uncovered in discovery calls of successful deals?', 'context': 'Relevant Context:\nThe pain points, objections, time lines, success indicators, budget constraints, competition, and decision committee members are the pieces of information about the buyer that are uncovered in discovery calls of successful deals.'}
Running sub query What all piece of information have we learnt about the buyer from discovery till now?
Sub query context {'query': 'What all piece of information have we learnt about the buyer from discovery till now?', 'context': 'Relevant Context:\nThe buyer, Sophos, is struggling to provide effective training and support to IT professionals and cybersecurity teams. They are also finding it difficult to stay ahead of cybercrime and ransomware attacks. Additionally, they have exp

Running Queries:  12%|█▎        | 1/8 [00:09<01:05,  9.43s/it]

Trying to parse with Ast...
Running sub query What are the top priorities for the account to solve for?


...

Overriding of current TracerProvider is not allowed


Sub query context {'query': 'What are the historical ways to present the product value prop to similar buyers', 'context': "Relevant Context:\nBy highlighting how the product can address specific pain points identified during the call, showcasing success indicators that align with the buyer's goals, providing insights into how the product has helped similar companies in the past, offering objection resolutions that address concerns raised by the buyer, and emphasizing the differentiation of the product from competitors in a way that resonates with the buyer's needs and industry challenges."}
Running final query Given stakeholder priorities and historical ways we've presented product to similar buyers - How can I present the value proposition of this product that aligns with my prospects industry and current stakeholders priorities?


Running Queries:  38%|███▊      | 3/8 [00:21<00:35,  7.07s/it]

Trying to parse with Ast...
Running sub query What are the top relevant discovery questions that have been successful for similar buyers and the respective stakeholder?


KeyboardInterrupt: 

In [4]:
responses["Info To Cover"]

{'response': "# Call Preparation Document\n## Discovery Call with Shell\n### Overview\nThe goal of this discovery call is to uncover the needs and pain points of Shell, a potential buyer, and understand how Whatfix can help them address their challenges.\n\n### Information Uncovered so far\n#### Challenges and Pain Points\n* Maximize technology investments and driving user adoption\n* Balancing oil demand with climate goals\n* Slow user adoption of new tools, impacting productivity and efficiency\n* Integration with existing systems\n* High degree of customization to meet unique requirements\n* Concerns about cost and implementation timeline of a digital adoption platform\n\n#### Decision Committee Members\n* IT\n* Operations\n* Business Leaders\n* CEO Wael Sawan\n* Mark (Shell icon)\n* Shareholders\n\n### Missing Information to Uncover\n#### From Historical Data\n* Objections\n* Time lines\n* Success indicators\n* Budget constraints\n* Competition\n\n#### From Current Call\n* None, as

In [1]:
# from echo.query_executor import aget_query_response


# response, sub_queries_context = await aget_query_response(
#     query,
#     inputs,
#     response_format=ResponseFormat.MARKDOWN, context_extraction_mode=ContextExtractionMode.QUERY_ENGINE
# )

In [ ]:
from echo.step_templates.discovery import aget_research_data_for_client
from echo.utils import get_llm
import nest_asyncio
import asyncio

nest_asyncio.apply()



data = asyncio.run(aget_research_data_for_client(inputs, llm=get_llm()))